In [89]:
import prepare as p
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

Work with the titanic dataset to answer the following:

In [90]:
titanic = p.prep_titanic()
print(titanic.shape)
titanic.head(3)

(891, 15)


,passenger_id,survived,sex,age,sibsp,parch,fare,class,embark_town,alone,embark_town_queenstown,embark_town_southampton,class_second,class_third,sex_male
0,0,0,male,22.0,1,0,7.2500,Third,Southampton,0,0,1,0,1,1
1,1,1,female,38.0,1,0,71.2833,First,Cherbourg,0,0,0,0,0,0
2,2,1,female,26.0,0,0,7.9250,Third,Southampton,1,0,1,0,1,0


Split data 70/20/10

In [118]:
train, validate, test = p.split_data(df=titanic, test_size=.1, validate_size=.2, 
                                     stratify_col='survived', random_state=25)
print(len(train), len(validate), len(test))
print(train['survived'].mean(), validate['survived'].mean(), test['survived'].mean())

623 178 90
0.38362760834670945 0.38202247191011235 0.3888888888888889


1. Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)


Making x-y split

In [92]:
def x_y_split(data_set, target, features):
    X = data_set[features]
    y = data_set[target]
    return X, y

In [93]:
train.columns[2:]

Index(['sex', 'age', 'sibsp', 'parch', 'fare', 'class', 'embark_town', 'alone',
       'embark_town_queenstown', 'embark_town_southampton', 'class_second',
       'class_third', 'sex_male'],
      dtype='object')

In [94]:
features = ['sibsp', 'parch', 'fare', 'alone', 'embark_town_queenstown', 
            'embark_town_southampton', 'class_second', 'class_third', 'sex_male']

X_train, y_train = x_y_split(data_set=train, target='survived', features=features)

X_validate, y_validate = x_y_split(data_set=validate, target='survived', features=features)

X_test, y_test = x_y_split(data_set=test, target='survived', features=features)

In [95]:
print(len(X_train),len(X_validate),len(X_test))

623 178 90


In [96]:
display(X_train.head(3), y_train.head(3))

,sibsp,parch,fare,alone,embark_town_queenstown,embark_town_southampton,class_second,class_third,sex_male
304,0,0,8.0500,1,0,1,0,1,1
171,4,1,29.1250,0,1,0,0,1,1
524,0,0,7.2292,1,0,0,0,1,1


304    0
171    0
524    0
Name: survived, dtype: int64

Baseline prediction

In [120]:
baseline_pred = y_train.mode()
baseline_pred.unique()

array([0])

In [ ]:
def establish_baseline(y_train):
    baseline_pred = y_train.mode()
    
    y_train_pred = pd.Series(baseline_pred[0])

Creating knn object

In [97]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

Fitting knn

In [98]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

Make predictions

In [101]:
y_pred_train = knn.predict(X_train)
y_pred_val = knn.predict(X_validate)

2. Evaluate your results using the model score, confusion matrix, and classification report.

In [122]:
print(f'Model score on training set: {knn.score(X_train, y_train)}')
print(f'Confusion Matrix on training set:\n {metrics.confusion_matrix(y_train, y_pred_train)}')
print(f'Classification Report on training set:\n {metrics.classification_report(y_train, y_pred_train)}')

Model score on training set: 0.7672552166934189
Confusion Matrix on training set:
 [[343  41]
 [104 135]]
Classification Report on training set:
               precision    recall  f1-score   support

           0       0.77      0.89      0.83       384
           1       0.77      0.56      0.65       239

    accuracy                           0.77       623
   macro avg       0.77      0.73      0.74       623
weighted avg       0.77      0.77      0.76       623



In [103]:
print(f'Model score on validation set: {knn.score(X_validate, y_validate)}')
print(f'Confusion Matrix on validation set:\n {metrics.confusion_matrix(y_validate, y_pred_val)}')
print(f'Classification Report on validation set:\n {metrics.classification_report(y_validate, y_pred_val)}')

Model score on validation set: 0.7191011235955056
Confusion Matrix on validation set:
 [[86 24]
 [26 42]]
Classification Report on validation set:
               precision    recall  f1-score   support

           0       0.77      0.78      0.77       110
           1       0.64      0.62      0.63        68

    accuracy                           0.72       178
   macro avg       0.70      0.70      0.70       178
weighted avg       0.72      0.72      0.72       178



3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

4. Run through steps 1-3 setting k to 10
5. Run through steps 1-3 setting k to 20

In [116]:
scores = []

for k in [1,3,5,10,15,20]:
    # Make KNN classifier 
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # Fit KNN classifier
    knn.fit(X_train, y_train)
    
    # Predict
    y_pred_train = knn.predict(X_train)
    y_pred_val = knn.predict(X_validate)
    
    output = {
        'k': k,
        'train_accuracy': metrics.accuracy_score(y_train, y_pred_train),
        'train_precision': metrics.precision_score(y_train, y_pred_train),
        'train_recall/TPR': metrics.recall_score(y_train, y_pred_train),
        'train_f1': metrics.f1_score(y_train, y_pred_train),
        'validate_accuracy':  metrics.accuracy_score(y_validate, y_pred_val),
        'validate_precision': metrics.precision_score(y_validate, y_pred_val),
        'validate_recall/TPR': metrics.recall_score(y_validate, y_pred_val),
        'validate_f1': metrics.f1_score(y_validate, y_pred_val),
    }

    scores.append(output)

baseline_preds_train = np.zeros(len(y_train))
baseline_preds_val = np.zeros(len(y_validate))
output = {
    'k': 'baseline',
    'train_accuracy': metrics.accuracy_score(y_train, baseline_preds_train),
    'train_precision': metrics.precision_score(y_train, baseline_preds_train),
    'train_recall/TPR': metrics.recall_score(y_train, baseline_preds_train),
    'train_f1': metrics.f1_score(y_train, baseline_preds_train),
    'validate_accuracy':  metrics.accuracy_score(y_validate, baseline_preds_val),
    'validate_precision': metrics.precision_score(y_validate, baseline_preds_val),
    'validate_recall/TPR': metrics.recall_score(y_validate, baseline_preds_val),
    'validate_f1': metrics.f1_score(y_validate, baseline_preds_val),
}

scores.append(output)

pd.DataFrame(scores)

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
0,1,0.916533,0.894515,0.887029,0.890756,0.747191,0.657534,0.705882,0.680851
1,3,0.863563,0.846847,0.786611,0.815618,0.735955,0.652174,0.661765,0.656934
2,5,0.826645,0.796380,0.736402,0.765217,0.719101,0.636364,0.617647,0.626866
3,10,0.780096,0.777174,0.598326,0.676123,0.724719,0.679245,0.529412,0.595041
4,15,0.768860,0.741117,0.610879,0.669725,0.713483,0.660377,0.514706,0.578512
5,20,0.767255,0.767045,0.564854,0.650602,0.724719,0.711111,0.470588,0.566372
6,baseline,0.616372,0.000000,0.000000,0.000000,0.617978,0.000000,0.000000,0.000000


6. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?


7. Which model performs best on our out-of-sample data from validate?


In [184]:
def x_y_split(data_set, target, features):
    X = data_set[features]
    y = data_set[target]
    return X, y

Get combination of desired training features

In [129]:
titanic.columns

Index(['passenger_id', 'survived', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'class', 'embark_town', 'alone', 'embark_town_queenstown',
       'embark_town_southampton', 'class_second', 'class_third', 'sex_male'],
      dtype='object')

In [139]:
import itertools

# Define the number of columns to select in each combination
k = len(titanic.columns)

# Get the column names from the dataset
model_columns = titanic.columns

# Generate combinations of column names
column_combinations = list(itertools.combinations(model_columns, k))

# Print the column combinations
for combination in column_combinations:
    print(combination)

('passenger_id', 'survived', 'sex', 'age', 'sibsp', 'parch', 'fare', 'class', 'embark_town', 'alone', 'embark_town_queenstown', 'embark_town_southampton', 'class_second', 'class_third', 'sex_male')


In [186]:
strong_model_columns = ['class_second', 'class_third','sex_male','fare','alone','embark_town_southampton']
all_model_columns = ['fare', 'alone', 'embark_town_queenstown', 'embark_town_southampton',
                     'class_second', 'class_third', 'sex_male']

get column combos

In [234]:
strong_column_combinations = []
for k in range(1, len(strong_model_columns)+1):
    for combo in list(itertools.combinations(strong_model_columns, k)):
        strong_column_combinations.append(list(combo))
len(strong_column_combinations)

63

In [235]:
all_column_combinations = []
for k in range(1, len(all_model_columns)+1):
    for combo in list(itertools.combinations(all_model_columns, k)):
        all_column_combinations.append(list(combo))
len(all_column_combinations)

127

In [237]:
def run_baseline_model(train, validate, target, model_columns):
    
    # split the data
    X_train, y_train = x_y_split(data_set=train, target=target, features=model_columns)

    X_validate, y_validate = x_y_split(data_set=validate, target=target, features=model_columns)
    
    # create baseline model
    baseline_pred = y_train.mode()
    y_train_pred = pd.Series((baseline_pred[0]), range(len(y_train)))
    y_val_pred = pd.Series((baseline_pred[0]), range(len(y_validate)))

    scores = []

    output = {
    'features': features,
    'model': 'baseline',
    'train_accuracy': metrics.accuracy_score(y_train, y_train_pred),
    'train_precision': metrics.precision_score(y_train, y_train_pred),
    'train_recall/TPR': metrics.recall_score(y_train, y_train_pred),
    'train_f1': metrics.f1_score(y_train, y_train_pred),
    'validate_accuracy':  metrics.accuracy_score(y_validate, y_val_pred),
    'validate_precision': metrics.precision_score(y_validate, y_val_pred),
    'validate_recall/TPR': metrics.recall_score(y_validate, y_val_pred),
    'validate_f1': metrics.f1_score(y_validate, y_val_pred),
    }
    scores.append(output)
    
    scores_df = pd.DataFrame(scores)
    
    return scores_df

In [238]:
display(run_baseline_model(train, validate, 'survived', strong_model_columns))

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,features,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
0,"[class_second, class_third, sex_male, fare, al...",baseline,0.616372,0.0,0.0,0.0,0.617978,0.0,0.0,0.0


In [296]:
def run_knn_model(train, validate, target, model_columns, k_range=range(1,21)):   
    '''
    takes in train set, validate set, target, columns to model
    gets all column combinations and runs knn models for each combination
    hyperparameter, k, is tuned by running each k in specified range, k_range
    scores for each model are stored in a DataFrame, scores_df
    
    Returns scores_df, a DataFrame of model scores
    '''
    # get column combinations
    column_combinations = []
    for k in range(1, len(model_columns)+1):
        for combo in list(itertools.combinations(model_columns, k)):
            column_combinations.append(list(combo))
    
    # for each combo of features
    for i, features in enumerate(column_combinations):
        
        # split X and y
        X_train, y_train = x_y_split(data_set=train, target=target, features=features)

        X_validate, y_validate = x_y_split(data_set=validate, target=target, features=features)
        
        scores = []
        
        # hyperparameter tuning
        for k in k_range:
            # Make KNN classifier 
            knn = KNeighborsClassifier(n_neighbors=k)

            # Fit KNN classifier
            knn.fit(X_train, y_train)

            # Predict
            y_pred_train = knn.predict(X_train)
            y_pred_val = knn.predict(X_validate)

            output = {
                'features': features,
                'model': f'k, k={k}',
                'train_accuracy': metrics.accuracy_score(y_train, y_pred_train),
                'train_precision': metrics.precision_score(y_train, y_pred_train),
                'train_recall/TPR': metrics.recall_score(y_train, y_pred_train),
                'train_f1': metrics.f1_score(y_train, y_pred_train),
                'validate_accuracy':  metrics.accuracy_score(y_validate, y_pred_val),
                'validate_precision': metrics.precision_score(y_validate, y_pred_val),
                'validate_recall/TPR': metrics.recall_score(y_validate, y_pred_val),
                'validate_f1': metrics.f1_score(y_validate, y_pred_val),
            }
            scores.append(output)
            
        # initialize scores DataFrame for first iteration
        if i == 0:
            scores_df = pd.DataFrame(scores)
        # concat scores DataFrames for the rest of the iterations
        else:
            scores_df = pd.concat([scores_df, pd.DataFrame(scores)])
            
    return scores_df

In [297]:
scores_df = run_knn_model(train, validate, 'survived', strong_model_columns)

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

In [ ]:
best_models = scores_df.sort_values(by='validate_f1', ascending=False).head(2)
best_models

In [295]:
for i in range(2):
    row = best_models.iloc[i]
    print(f'''features: {row['features']}, k: {row['k']}, f1_score: {row['validate_f1']}''')

features: ['class_second', 'sex_male', 'fare', 'embark_town_southampton'], k: 1, f1_score: 0.7313432835820897
features: ['class_second', 'sex_male', 'fare', 'embark_town_southampton'], k: 3, f1_score: 0.7230769230769231


<div style="border: 3px solid black;"></div>
    
Extra Functions

Create baseline and knn model

In [189]:
features =  ['parch',
             'fare',
             'alone',
             'embark_town_queenstown',
             'embark_town_southampton',
             'class_third',
             'sex_male']

# split the data
train, validate, test = p.split_data(df=titanic, test_size=.1, validate_size=.2, 
                                     stratify_col='survived', random_state=25)

# x-y split the data
X_train, y_train = x_y_split(data_set=train, target='survived', features=features)

X_validate, y_validate = x_y_split(data_set=validate, target='survived', features=features)

X_test, y_test = x_y_split(data_set=test, target='survived', features=features)

# create baseline model
baseline_pred = y_train.mode()
y_train_pred = pd.Series((baseline_pred[0]), range(len(y_train)))
y_val_pred = pd.Series((baseline_pred[0]), range(len(y_validate)))

scores = []

output = {
    'k': 'baseline',
    'train_accuracy': metrics.accuracy_score(y_train, y_train_pred),
    'train_precision': metrics.precision_score(y_train, y_train_pred),
    'train_recall/TPR': metrics.recall_score(y_train, y_train_pred),
    'train_f1': metrics.f1_score(y_train, y_train_pred),
    'validate_accuracy':  metrics.accuracy_score(y_validate, y_val_pred),
    'validate_precision': metrics.precision_score(y_validate, y_val_pred),
    'validate_recall/TPR': metrics.recall_score(y_validate, y_val_pred),
    'validate_f1': metrics.f1_score(y_validate, y_val_pred),
}

scores.append(output)


# create knn model
knn = KNeighborsClassifier(n_neighbors=5)


for k in [1,3,5,10,15,20]:
    # Make KNN classifier 
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # Fit KNN classifier
    knn.fit(X_train, y_train)
    
    # Predict
    y_pred_train = knn.predict(X_train)
    y_pred_val = knn.predict(X_validate)
    
    output = {
        'k': k,
        'train_accuracy': metrics.accuracy_score(y_train, y_pred_train),
        'train_precision': metrics.precision_score(y_train, y_pred_train),
        'train_recall/TPR': metrics.recall_score(y_train, y_pred_train),
        'train_f1': metrics.f1_score(y_train, y_pred_train),
        'validate_accuracy':  metrics.accuracy_score(y_validate, y_pred_val),
        'validate_precision': metrics.precision_score(y_validate, y_pred_val),
        'validate_recall/TPR': metrics.recall_score(y_validate, y_pred_val),
        'validate_f1': metrics.f1_score(y_validate, y_pred_val),
    }
    scores.append(output)
    
pd.DataFrame(scores)

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
0,baseline,0.616372,0.000000,0.000000,0.000000,0.617978,0.000000,0.000000,0.000000
1,1,0.913323,0.887029,0.887029,0.887029,0.752809,0.666667,0.705882,0.685714
2,3,0.855538,0.843318,0.765690,0.802632,0.769663,0.701493,0.691176,0.696296
3,5,0.841091,0.809735,0.765690,0.787097,0.747191,0.682540,0.632353,0.656489
4,10,0.773676,0.778409,0.573222,0.660241,0.724719,0.693878,0.500000,0.581197
5,15,0.759230,0.737968,0.577406,0.647887,0.713483,0.660377,0.514706,0.578512
6,20,0.751204,0.750000,0.527197,0.619165,0.702247,0.653061,0.470588,0.547009


Generalize to run on combination of columns

In [236]:
# split the data
train, validate, test = p.split_data(df=titanic, test_size=.1, validate_size=.2, 
                                     stratify_col='survived', random_state=25)

scores_df = {}

for features in strong_column_combinations:
    # split the data
    X_train, y_train = x_y_split(data_set=train, target='survived', features=features)
    
    X_validate, y_validate = x_y_split(data_set=validate, target='survived', features=features)

    X_test, y_test = x_y_split(data_set=test, target='survived', features=features)
    
    # create baseline model
    baseline_pred = y_train.mode()
    y_train_pred = pd.Series((baseline_pred[0]), range(len(y_train)))
    y_val_pred = pd.Series((baseline_pred[0]), range(len(y_validate)))

    scores = []
    
    output = {
    'k': 'baseline',
    'train_accuracy': metrics.accuracy_score(y_train, y_train_pred),
    'train_precision': metrics.precision_score(y_train, y_train_pred),
    'train_recall/TPR': metrics.recall_score(y_train, y_train_pred),
    'train_f1': metrics.f1_score(y_train, y_train_pred),
    'validate_accuracy':  metrics.accuracy_score(y_validate, y_val_pred),
    'validate_precision': metrics.precision_score(y_validate, y_val_pred),
    'validate_recall/TPR': metrics.recall_score(y_validate, y_val_pred),
    'validate_f1': metrics.f1_score(y_validate, y_val_pred),
    }
    scores.append(output)
    
    # hyperparameter tuning
    for k in range(1,21):
        # Make KNN classifier 
        knn = KNeighborsClassifier(n_neighbors=k)

        # Fit KNN classifier
        knn.fit(X_train, y_train)

        # Predict
        y_pred_train = knn.predict(X_train)
        y_pred_val = knn.predict(X_validate)

        output = {
            'k': k,
            'train_accuracy': metrics.accuracy_score(y_train, y_pred_train),
            'train_precision': metrics.precision_score(y_train, y_pred_train),
            'train_recall/TPR': metrics.recall_score(y_train, y_pred_train),
            'train_f1': metrics.f1_score(y_train, y_pred_train),
            'validate_accuracy':  metrics.accuracy_score(y_validate, y_pred_val),
            'validate_precision': metrics.precision_score(y_validate, y_pred_val),
            'validate_recall/TPR': metrics.recall_score(y_validate, y_pred_val),
            'validate_f1': metrics.f1_score(y_validate, y_pred_val),
        }
        scores.append(output)

        scores_df[str(features)] = pd.DataFrame(scores)

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

In [217]:
for k, df in scores_df.items():
    print(f'{k}:')
    display(df.sort_values(by='validate_accuracy', ascending=False))

['class_second']:


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
0,baseline,0.616372,0.000000,0.000000,0.000000,0.617978,0.000000,0.000000,0.000000
4,4,0.616372,0.000000,0.000000,0.000000,0.617978,0.000000,0.000000,0.000000
20,20,0.616372,0.000000,0.000000,0.000000,0.617978,0.000000,0.000000,0.000000
2,2,0.616372,0.000000,0.000000,0.000000,0.617978,0.000000,0.000000,0.000000
3,3,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424
19,19,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424
18,18,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424
17,17,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424
16,16,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424
15,15,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424


['class_third']:


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
10,10,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
11,11,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
17,17,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
15,15,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
14,14,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
13,13,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
9,9,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
12,12,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
19,19,0.616372,0.000000,0.00000,0.000000,0.617978,0.000000,0.000000,0.000000
18,18,0.616372,0.000000,0.00000,0.000000,0.617978,0.000000,0.000000,0.000000


['sex_male']:


...

,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
3,3,0.863563,0.846847,0.786611,0.815618,0.786517,0.734375,0.691176,0.712121
7,7,0.821830,0.799065,0.715481,0.754967,0.775281,0.741379,0.632353,0.682540
1,1,0.908507,0.888889,0.870293,0.879493,0.769663,0.695652,0.705882,0.700730
4,4,0.829856,0.863388,0.661088,0.748815,0.769663,0.736842,0.617647,0.672000
5,5,0.839486,0.817352,0.748954,0.781659,0.758427,0.698413,0.647059,0.671756
6,6,0.828250,0.854839,0.665272,0.748235,0.758427,0.735849,0.573529,0.644628
10,10,0.788122,0.795580,0.602510,0.685714,0.752809,0.730769,0.558824,0.633333
13,13,0.796148,0.774510,0.661088,0.713318,0.752809,0.706897,0.602941,0.650794
8,8,0.794543,0.803279,0.615063,0.696682,0.752809,0.722222,0.573529,0.639344
9,9,0.807384,0.796020,0.669456,0.727273,0.747191,0.709091,0.573529,0.634146


['class_third', 'sex_male', 'fare', 'alone', 'embark_town_southampton']:


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
3,3,0.868379,0.852018,0.794979,0.822511,0.786517,0.734375,0.691176,0.712121
7,7,0.826645,0.804651,0.723849,0.762115,0.769663,0.728814,0.632353,0.677165
1,1,0.908507,0.888889,0.870293,0.879493,0.769663,0.695652,0.705882,0.700730
4,4,0.841091,0.868421,0.690377,0.769231,0.764045,0.724138,0.617647,0.666667
5,5,0.849117,0.819383,0.778243,0.798283,0.758427,0.698413,0.647059,0.671756
6,6,0.834671,0.869565,0.669456,0.756501,0.752809,0.722222,0.573529,0.639344
8,8,0.804173,0.816216,0.631799,0.712264,0.752809,0.722222,0.573529,0.639344
13,13,0.796148,0.766667,0.673640,0.717149,0.752809,0.700000,0.617647,0.656250
10,10,0.789727,0.796703,0.606695,0.688836,0.747191,0.716981,0.558824,0.628099
9,9,0.808989,0.800000,0.669456,0.728929,0.747191,0.709091,0.573529,0.634146


['class_second', 'class_third', 'sex_male', 'fare', 'alone', 'embark_town_southampton']:


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
3,3,0.863563,0.846847,0.786611,0.815618,0.780899,0.730159,0.676471,0.702290
1,1,0.908507,0.888889,0.870293,0.879493,0.769663,0.695652,0.705882,0.700730
5,5,0.841091,0.812500,0.761506,0.786177,0.758427,0.698413,0.647059,0.671756
7,7,0.815409,0.800971,0.690377,0.741573,0.758427,0.727273,0.588235,0.650407
4,4,0.833066,0.860963,0.673640,0.755869,0.752809,0.714286,0.588235,0.645161
11,11,0.796148,0.764151,0.677824,0.718404,0.752809,0.700000,0.617647,0.656250
13,13,0.797753,0.772947,0.669456,0.717489,0.752809,0.700000,0.617647,0.656250
10,10,0.799358,0.813187,0.619247,0.703088,0.747191,0.716981,0.558824,0.628099
19,19,0.764045,0.761364,0.560669,0.645783,0.741573,0.729167,0.514706,0.603448
6,6,0.825040,0.861111,0.648536,0.739857,0.741573,0.711538,0.544118,0.616667


Make generalized code into a function

In [214]:
# split the data
# train, validate, test = p.split_data(df=titanic, test_size=test_size, validate_size=validate_size, 
#                                          stratify_col=stratify_col, random_state=random_state)


def run_knn_model(train, validate, target, model_columns):   
    '''
    takes in train set, validate set, target, columns to model
    '''
    # get column combinations
    column_combinations = []
    for k in range(1, len(model_columns)+1):
        for combo in list(itertools.combinations(model_columns, k)):
            column_combinations.append(list(combo))
    
    scores_df = {}

    for features in column_combinations:
        # split the data
        X_train, y_train = x_y_split(data_set=train, target=target, features=features)

        X_validate, y_validate = x_y_split(data_set=validate, target=target, features=features)

        X_test, y_test = x_y_split(data_set=test, target=target, features=features)

        # create baseline model
        baseline_pred = y_train.mode()
        y_train_pred = pd.Series((baseline_pred[0]), range(len(y_train)))
        y_val_pred = pd.Series((baseline_pred[0]), range(len(y_validate)))

        scores = []

        output = {
        'k': 'baseline',
        'train_accuracy': metrics.accuracy_score(y_train, y_train_pred),
        'train_precision': metrics.precision_score(y_train, y_train_pred),
        'train_recall/TPR': metrics.recall_score(y_train, y_train_pred),
        'train_f1': metrics.f1_score(y_train, y_train_pred),
        'validate_accuracy':  metrics.accuracy_score(y_validate, y_val_pred),
        'validate_precision': metrics.precision_score(y_validate, y_val_pred),
        'validate_recall/TPR': metrics.recall_score(y_validate, y_val_pred),
        'validate_f1': metrics.f1_score(y_validate, y_val_pred),
        }
        scores.append(output)

        # hyperparameter tuning
        for k in range(1,21):
            # Make KNN classifier 
            knn = KNeighborsClassifier(n_neighbors=k)

            # Fit KNN classifier
            knn.fit(X_train, y_train)

            # Predict
            y_pred_train = knn.predict(X_train)
            y_pred_val = knn.predict(X_validate)

            output = {
                'k': k,
                'train_accuracy': metrics.accuracy_score(y_train, y_pred_train),
                'train_precision': metrics.precision_score(y_train, y_pred_train),
                'train_recall/TPR': metrics.recall_score(y_train, y_pred_train),
                'train_f1': metrics.f1_score(y_train, y_pred_train),
                'validate_accuracy':  metrics.accuracy_score(y_validate, y_pred_val),
                'validate_precision': metrics.precision_score(y_validate, y_pred_val),
                'validate_recall/TPR': metrics.recall_score(y_validate, y_pred_val),
                'validate_f1': metrics.f1_score(y_validate, y_pred_val),
            }
            scores.append(output)

            scores_df[str(features)] = pd.DataFrame(scores)
            
    return scores_df

In [215]:
scores_df = run_knn_model(train, validate, 'survived', strong_model_columns)

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

In [222]:
for k, df in scores_df.items():
    print(f'{k}:')
    display(df.sort_values(by='validate_accuracy', ascending=False).head(3))

['class_second']:


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
0,baseline,0.616372,0.0,0.0,0.0,0.617978,0.0,0.0,0.0
4,4,0.616372,0.0,0.0,0.0,0.617978,0.0,0.0,0.0
20,20,0.616372,0.0,0.0,0.0,0.617978,0.0,0.0,0.0


['class_third']:


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
10,10,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
11,11,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529
17,17,0.672552,0.560976,0.67364,0.612167,0.674157,0.573529,0.573529,0.573529


['sex_male']:


...

,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
3,3,0.863563,0.846847,0.786611,0.815618,0.786517,0.734375,0.691176,0.712121
7,7,0.821830,0.799065,0.715481,0.754967,0.775281,0.741379,0.632353,0.682540
1,1,0.908507,0.888889,0.870293,0.879493,0.769663,0.695652,0.705882,0.700730


['class_third', 'sex_male', 'fare', 'alone', 'embark_town_southampton']:


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
3,3,0.868379,0.852018,0.794979,0.822511,0.786517,0.734375,0.691176,0.712121
7,7,0.826645,0.804651,0.723849,0.762115,0.769663,0.728814,0.632353,0.677165
1,1,0.908507,0.888889,0.870293,0.879493,0.769663,0.695652,0.705882,0.700730


['class_second', 'class_third', 'sex_male', 'fare', 'alone', 'embark_town_southampton']:


,k,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
3,3,0.863563,0.846847,0.786611,0.815618,0.780899,0.730159,0.676471,0.702290
1,1,0.908507,0.888889,0.870293,0.879493,0.769663,0.695652,0.705882,0.700730
5,5,0.841091,0.812500,0.761506,0.786177,0.758427,0.698413,0.647059,0.671756
